In [ ]:
import pandas as pd 

In [ ]:
titanic = pd.read_csv(r"C:\Users\Casper\Desktop\pandada\titanic.csv")

In [ ]:
air_quality = pd.read_csv(
    r"C:\Users\Casper\Desktop\pandada\air_quality_long.csv", index_col="date.utc", parse_dates=True
    )

air_quality.head()

## Note:
The air-quality data is provided in a so-called long format data representation with each observation on a seperate row and each variable a seperate column of the data table.

# How to reshape the layout of tables 

## Sort table rows

In [ ]:
# I want to sort the Titanic data according to the age of the passengers. 
titanic.sort_values(by="Age").head()

In [ ]:
# I want to sort the Titanic data according to the cabin class and age in descending order. 
titanic.sort_values(by=['Pclass', 'Age'], ascending=False).head()

## Long to wide table format 
Let's use a small subset of the air quality data set. We focus on NO2 data and only use the first two measurements of each location. The subset of data will be called no2_subset. 

In [ ]:
# filter for no2 data only 
no2 = air_quality[air_quality["parameter"] == "no2"]

In [ ]:
# use 2 measurement (head) for each location (groupby)
no2_subset = no2.sort_index().groupby(["location"]).head(2)

no2_subset

In [ ]:
# I want the values for three stations as seperate columns next to each other. 
no2_subset.pivot(columns="location", values="value")

# The pivot() function is purely reshaping of the data: a single value for each index/column combination is required. 

As pandas supports plotting of multiple columns out of the box, the conversion from long to wide table format enables the plotting of the different time series at the same time. 

In [ ]:
no2.head()

In [ ]:
no2.pivot(columns="location", values="value").plot()

In [ ]:
# I want the mean concentrations for NO2 and PM2.5 in each of the stations in table form. 
air_quality.pivot_table(
    values="value", index="location", columns="parameter", aggfunc="mean"
)

In this case of pivot(), the data is only rearranged. When multiple values need to be aggregated, 
pivot_table() can be used, providing an aggregation function on how to combine these values. 

Pivot table is a well known concept in spreadsheet software. When interested in the row/column margins (subtotals) for each variable, set the margins parameter to True:

In [ ]:
air_quality.pivot_table(
    values="value",
    index="location",
    columns="parameter",
    aggfunc="mean",
    margins=True,
)

## Note: 
In case we are wondering, pivot_table() is indeed directly linked to groupby(). The same result can be derived by grouping on both parameter and location: 

In [ ]:
air_quality.groupby(["parameter", "location"])[["value"]].mean()

## Wide to long format 
Starting again from the wide format table created in the previous section, we add a new index to the DataFrame with reset_index(). 

In [ ]:
no2_pivoted = no2.pivot(columns="location", values="value").reset_index()

no2_pivoted.head()

In [ ]:
# I want to collec all air quality NO2 measurements in a single column (long format). 
no_2 = no2_pivoted.melt(id_vars="date.utc")

no_2.head()

The pandas.melt() method on a DataFrame converts the data table from wide format to long format. The column headers become the variable names in a newly created column. 

The solution is the short version on how to apply pandas.melt(). The method will melt all columns NOT mentioned in id_vars together into two columns: A column with the column header names and a column with the values itself. The latter column gets by default the name value. 

The parameter passed to pandas.melt() can be defined in more detail:

In [ ]:
no_2 = no2_pivoted.melt(
    id_vars="date.utc", 
    value_vars=["BETR801", "FR04014", "London Westminster"],
    value_name="NO_2",
    var_name="id_location",
)

no_2.head()

The additional parameters have the following effects: 
* value_vars defines which column to melt together 
* value_name provides a custom column name for the values column instead of the default column name value 
* var_name provides a custom column name for the column collecting the column header names. Otherwise it takes the index name or a default variable. 

Hence, the arguments value_name and var_name are just user-defined names for the two generated columns. The columns to mlet are defined by id_vars and value_vars. 

## Remember
* Sorting by one or more column is supported by sort_values. 
* The pivot function is purely restructing of the data, pivot_table supports aggregations. 
* The reverse of pivot (long to wide format) is metl (wide to long format).  